Insper

# Aula 03 - Práticas PySpak

## Configurando Spark localmente

Instalando o Spark na versão 3.1.3 no Colab. Eventualmente você precisará reiniciar seu ambiente.

In [ ]:
!pip install pyspark==3.1.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 14.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.3-py2.py3-none-any.whl size=214463458 sha256=ebf6a3bdf85c7f6fae20f7116f520a6a1203fc37184ce54f26099fcfc8a59399
  Stored in directory: /root/.cache/pip/wheels/b0/d0/fd/b0e0165f0fbd79838d2f50c544382e7a5346274f0af07ffada
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.3
    Uninstalling pyspark-3.5.3:
      Successfully uninstalled pyspark-3.5.3


## Lab 1 - Cálculo do PI

A proporção de pontos internos ao círculo circunscrito no quadrado de raio=1 é igual à PI / 4

In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
        .master("local[*]")
        .appName("PI")
        .getOrCreate())

spark

In [2]:
from random import random

def circulo(_):
    x = random()
    y = random()

    if (x**2 + y**2) <= 1:
        return 1
    else:
        return 0

In [3]:
circulo(9)

1

In [4]:
circulo(9.25)

0

In [5]:
circulo("michel")

1

In [6]:
sc = spark.sparkContext

In [7]:
n_pontos = 20000000
pontos = sc.parallelize( range(0, n_pontos)  )

In [8]:
pontos.take(20)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [9]:
# O comando abaixo é exatamente o mesmo que: amostras = pontos.map(lambda x: circulo(x))
amostras = pontos.map(circulo)

In [10]:
amostras.take(20)

[1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1]

In [11]:
PI = (amostras.sum() / n_pontos) * 4
print(f"O valor aproximado de PI é {PI}")

O valor aproximado de PI é 3.1415832


In [12]:
import math
math.pi

3.141592653589793

## Lab 2 - Cálculos simples em RDD

Faça um programa em Spark que:

- Crie uma RDD com 30 inteiros
- Subtraia 1 de cada valor utilizando `map()`
- Realize o `collect()` para visualizar os resultados
- Realize o `count()` para verificar quantos registros existem na RDD
- Filtre valores abaixo de 10

In [13]:
n_points = 30
points = sc.parallelize(range(n_points))
points.take(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [14]:
rdd_subtracao = points.map(lambda x: x - 1)

In [15]:
rdd_subtracao.take(2)

[-1, 0]

In [16]:
print(rdd_subtracao.collect()) # CUIDADO COM O COLLECT()!!! Ele puxa tudo pra memória, e pode capotar o código!

[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]


In [18]:
rdd_subtracao.count()

30

In [19]:
rdd_subtracao.filter( lambda x: x < 10 ).collect()

[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## Lab 3 - Análise de dados (simples) do MovieLens

Vamos complicar um pouco as coisas! Agora vamos trabalhar com uma base de dados. Para isso, vamos baixar a base.

### Obtendo os dados


In [20]:
# Importing the movie lens dataset directly to colab
!wget --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-25m.zip

--2024-10-26 20:35:24--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M  32.6MB/s    in 8.4s    

2024-10-26 20:35:33 (29.8 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]



In [21]:
!unzip ml-25m.zip

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


### Setup

Criar uma sessão do Spark.

In [22]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .master("local[*]") \
        .appName("movielens_rdd") \
        .getOrCreate()

spark

Obtendo o `contexto` do Spark, onde encontramos as RDDs.

In [23]:
sc = spark.sparkContext

### A) Quantas avaliações existem na base?

Ou seja, quantas linhas de dados? O valor obtido era esperado mesmo? Por quê?

In [46]:
ratings_rdd = sc.textFile( "ml-25m/ratings.csv" )
ratings_rdd.count()

25000096

In [25]:
ratings_rdd.take(10)

['userId,movieId,rating,timestamp',
 '1,296,5.0,1147880044',
 '1,306,3.5,1147868817',
 '1,307,5.0,1147868828',
 '1,665,5.0,1147878820',
 '1,899,3.5,1147868510',
 '1,1088,4.0,1147868495',
 '1,1175,3.5,1147868826',
 '1,1217,3.5,1147878326',
 '1,1237,5.0,1147868839']

In [47]:
cabecalho = ratings_rdd.first()
cabecalho

'userId,movieId,rating,timestamp'

In [48]:
ratings_rdd = ratings_rdd.filter( lambda x: x != cabecalho )
ratings_rdd.take(10)

['1,296,5.0,1147880044',
 '1,306,3.5,1147868817',
 '1,307,5.0,1147868828',
 '1,665,5.0,1147878820',
 '1,899,3.5,1147868510',
 '1,1088,4.0,1147868495',
 '1,1175,3.5,1147868826',
 '1,1217,3.5,1147878326',
 '1,1237,5.0,1147868839',
 '1,1250,4.0,1147868414']

### B) Contagem de Avaliações por Filme

**Objetivo**: Encontre o número de avaliações que cada filme recebeu.

**Quer dica?**:
- Crie um RDD com (movieId, 1).
- Use reduceByKey para contar quantas avaliações cada filme recebeu.

In [49]:
# DICA: Para rodar mais rápido:
# - limite a quantidade de linhas
# - paralelize-as diretamente
# - programe e debugue normalmente!
# - após ter certeza que seu código funciona, comente as 2 próximas linhas e rode para todo o dataset
#selecao = ratings_rdd.take(1000)
#ratings_rdd = sc.parallelize(selecao)

movie_count = ratings_rdd.map( lambda x: (x.split(",")[1], 1) )
contagem = movie_count.reduceByKey(lambda a, b: a+b)
contagem.take(10)

[('3448', 12164),
 ('5269', 1079),
 ('6539', 37227),
 ('7318', 4330),
 ('260', 68717),
 ('914', 9692),
 ('1302', 12352),
 ('1376', 12585),
 ('1587', 7015),
 ('2115', 24306)]

In [56]:
contagem.takeOrdered(5, key=lambda x: -x[1])

[('356', 81491),
 ('318', 81482),
 ('296', 79672),
 ('593', 74127),
 ('2571', 72674)]

### C) Identificação de Filmes com Avaliação Mínima e Máxima

**Objetivo**: Encontre o filme com a _menor_ e a _maior_ nota.

**Quer dica?**:
- Crie um RDD com (movieId, rating).
- Use reduceByKey para encontrar o menor e o maior valor para cada filme.

In [51]:
movie_min_rating = ratings_rdd.map(lambda line: (line.split(',')[1], float(line.split(',')[2]))) \
                              .reduceByKey(lambda a, b: min(a, b))
movie_max_rating = ratings_rdd.map(lambda line: (line.split(',')[1], float(line.split(',')[2]))) \
                              .reduceByKey(lambda a, b: max(a, b))
print("Min:", movie_min_rating.take(10))
print("Max:", movie_max_rating.take(10))


Min: [('3448', 0.5), ('5269', 0.5), ('6539', 0.5), ('7318', 0.5), ('260', 0.5), ('914', 0.5), ('1302', 0.5), ('1376', 0.5), ('1587', 0.5), ('2115', 0.5)]
Max: [('3448', 5.0), ('5269', 5.0), ('6539', 5.0), ('7318', 5.0), ('260', 5.0), ('914', 5.0), ('1302', 5.0), ('1376', 5.0), ('1587', 5.0), ('2115', 5.0)]


### D) Número de Filmes Avaliados por Usuário

**Objetivo**: Encontre quantos filmes cada usuário avaliou

**Quer dica?**:
- Crie um RDD com (userId, 1) para contar as avaliações de cada usuário.
- Use reduceByKey para contar quantos filmes cada usuário avaliou.

In [52]:
user_movie_count = ratings_rdd.map(lambda line: (line.split(',')[0], 1)) \
                              .reduceByKey(lambda a, b: a + b)
user_movie_count.take(10)

[('2', 184),
 ('13', 412),
 ('24', 25),
 ('33', 23),
 ('70', 196),
 ('76', 182),
 ('77', 45),
 ('111', 23),
 ('113', 133),
 ('119', 124)]

In [53]:
user_movie_count.takeOrdered(10, key=lambda x: -x[1])

[('72315', 32202),
 ('80974', 9178),
 ('137293', 8913),
 ('33844', 7919),
 ('20055', 7488),
 ('109731', 6647),
 ('92046', 6564),
 ('49403', 6553),
 ('30879', 5693),
 ('115102', 5649)]

### E) Identificar Filmes Avaliados com Exatamente 5 Estrelas

**Objetivo**: Encontre filmes que receberam exatamente 5 estrelas

**Quer dica?**:
- Crie um RDD com (movieId, rating).
- Use filter para manter apenas as avaliações com 5 estrelas.
- Use distinct para obter os filmes únicos que receberam essa nota.

In [54]:
movies_with_5_stars = ratings_rdd.map(lambda line: (line.split(',')[1], float(line.split(',')[2]))) \
                                 .filter(lambda x: x[1] == 5.0) \
                                 .map(lambda x: x[0]) \
                                 .distinct()
print(movies_with_5_stars.take(10))


['260', '1376', '3360', '4995', '5816', '6539', '293', '1221', '5618', '48780']


### F) Cálculo da Nota Média por Filme

**Objetivo**: Dado um dataset com avaliações de filmes (userId, movieId, rating), calcule a nota média para cada filme.

**Quer dica?**:
- Crie um RDD a partir do arquivo de ratings.
- Utilize a função map para transformar cada linha no formato (movieId, rating).
- Use reduceByKey para calcular a soma e a contagem de avaliações por filme.
- Calcule a média dividindo a soma das notas pela contagem de notas. Pode ser útil usar o [mapValues()](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.mapValues.html).

In [55]:
movie_ratings = ratings_rdd.map(lambda line: (line.split(',')[1], (float(line.split(',')[2]), 1)))
movie_rating_totals = movie_ratings.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
movie_rating_avg = movie_rating_totals.mapValues(lambda total: total[0] / total[1])
print(movie_rating_avg.take(10))

[('3448', 3.666639263400197), ('5269', 3.572289156626506), ('6539', 3.7892121309802027), ('7318', 3.14445727482679), ('260', 4.120188599618726), ('914', 3.9047152290548905), ('1302', 3.63714378238342), ('1376', 3.5012713547874452), ('1587', 3.243478260869565), ('2115', 3.695939274253271)]


### Lab 5 - Trabalhando com texto

Vamos ver como usar RDDs para processar palavras!

In [37]:
text_rdd = sc.textFile( "texto.txt" )

In [38]:
text_rdd.take(2)

['O Apache Spark é uma ferramenta essencial no universo da ciência de dados devido à sua capacidade de processar grandes volumes de dados de maneira distribuída e eficiente. A sua arquitetura permite que operações de análise de dados sejam realizadas em clusters, possibilitando o processamento paralelo, o que acelera significativamente o tempo de execução de algoritmos complexos. Isso é especialmente relevante em cenários onde o volume de dados é muito grande para ser tratado por ferramentas tradicionais, como Pandas, que operam de forma local.',
 '']

Há quantas linhas no arquivo?

In [39]:
text_rdd.count()

19

Há quantas linhas com texto?

In [40]:
text_rdd.filter(lambda l: len(l) > 0).count()

10

Quantas vezes a palavra "Spark" apareceu no texto?

In [42]:
text_rdd.flatMap(lambda l: l.split(" ")).filter(lambda w: w == "Spark").count()

17

Quantas vezes cada palavra apareceu?

In [43]:
contagemPlavras = text_rdd.flatMap(lambda l: l.split(" ")) \
                .map(lambda w: (w, 1)) \
                .reduceByKey(lambda a, b: a+b)

In [44]:
contagemPlavras.take(10)

[('O', 7),
 ('Apache', 1),
 ('Spark', 17),
 ('uma', 9),
 ('no', 1),
 ('universo', 1),
 ('da', 2),
 ('dados', 17),
 ('à', 2),
 ('capacidade', 3)]

In [45]:
contagemPlavras.takeOrdered(10, key=lambda x: -x[1])

[('de', 61),
 ('o', 31),
 ('em', 19),
 ('que', 18),
 ('para', 18),
 ('e', 18),
 ('Spark', 17),
 ('dados', 17),
 ('é', 17),
 ('do', 14)]